# GirassolGPT🌻
Bem-vinda!

Agora que já temos a informação real dos vários componentes que afetam o crescimento da tua planta, como é que vamos monitorizar se temos as condições perfeitas para elas se desenvolverem? Vamos criar um ChatGPT que te permitirá perceber o que se passa com elas! Faz as perguntas sobre as condições das tuas plantas, se precisam de mais água, se precisam de mais sol, compara com as condições atmosféricas neste momento em Lisboa, etc.

## Prepara o ficheiro 
1. Se escolheste o girassol, dentro da pasta `mysunflower` adiciona o teu ficheiro CSV com os dados da tua planta.
2. Muda o nome do teu ficheiro para `mysunflower.csv`
3. No topo do teu ecrã, terás um botão que se chama "Run All". Clica neste botão.

Nos próximos passos, vais ver algum código. Não te preocupes, não precisas de programar! No entanto, precisas de fazer alguma coisa onde encontrares a palavra "Ação!"

In [10]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from dotenv import load_dotenv
from openai import AzureOpenAI
import json
from datetime import datetime, timedelta
from IPython.display import Markdown, display

load_dotenv()

True

De seguida, temos apenas um bocado de código que vai criar 3 gráficos que analisam as 3 variáveis sobre o solo da tua planta ao longo do dia:
- Temperatura
- Humidadade
- Luminosidade

Neste passo, não precisas de fazer nada.

In [11]:
import os
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from openai import AzureOpenAI
import json
from datetime import datetime, timedelta
from IPython.display import Markdown, display

load_dotenv()
df = pd.read_csv('mysunflower.csv')

#AÇÃO! Muda aqui os valores correspondentes à data de recolha dos dados
start_time = datetime(2025, 3, 17, 15, 0, 0)  # Year, month, day, hour, minute, second

df['exact_time'] = df['time (seconds)'].apply(lambda x: (start_time + timedelta(seconds=x)).strftime('%H:%M:%S'))
df.to_csv('mysunflower_with_time.csv', index=False)

In [ ]:
# Read the CSV file with the exacttime column
data = pd.read_csv('mysunflower_with_time.csv')

# Convert the exacttime column to datetime objects for proper plotting
data['exact_time'] = pd.to_datetime(data['exact_time'], format='%H:%M:%S')

# Create some basic visualizations
plt.figure(figsize=(12, 8))

# Format for time display
time_fmt = mdates.DateFormatter('%H:%M:%S')

plt.subplot(3, 1, 1)
plt.plot(data['exact_time'], data['temperature'], 'r-')
plt.title('Temperature over Time')
plt.ylabel('Temperature')
plt.gca().xaxis.set_major_formatter(time_fmt)
plt.xticks(rotation=45)

plt.subplot(3, 1, 2)
plt.plot(data['exact_time'], data['moisture'], 'b-')
plt.title('Moisture over Time')
plt.ylabel('Moisture')
plt.gca().xaxis.set_major_formatter(time_fmt)
plt.xticks(rotation=45)

plt.subplot(3, 1, 3)
plt.plot(data['exact_time'], data['light'], 'g-')
plt.title('Light over Time')
plt.ylabel('Light')
plt.gca().xaxis.set_major_formatter(time_fmt)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

ValueError: time data "15:00:01" doesn't match format "%Y-%m-%d %H:%M:%S", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

Agora chegaste à parte mais gira... a criação do SunflowerGPT! O código que vês aqui serve apenas de apoio. Aquilo onde precisas de te focar é no texto que está entre aspas triplas e se chama `system_message`. A mensagem de sistema, serve para tu dizeres ao teu SunflowerGPT como se deve comportar. No texto abaixo consegues já encontrar algumas ideias de como um chat se comporta. Carrega no botão "Run" e experimenta esta versão, de seguida voltarás a este código para trazeres mais ideias giras de coisas a implementar.

In [ ]:
# Create Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2023-12-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Prepare system message with data context
data_summary = data.describe().to_string()
data_sample = data.head(10).to_string()
with open("aboutsunflowers.txt", "r", encoding="utf-8") as file:
    aboutsunflower = file.read()

system_message = f"""
És um assistente de IA agrícola especializado em horticultura e monitorização da saúde das plantas. 
Tens acesso aos seguintes dados de sensores de um dispositivo micro:bit que monitoriza uma horta:

RESUMO DOS DADOS DO SENSOR:
{data_summary}

INFORMAÇÕES IMPORTANTES SOBRE O GIRASSOL:
{aboutsunflower}

DADOS CLIMÁTICOS:
São 15h00 em Portugal, em março, e o tempo está ensolarado com uma temperatura de 20°C. A humidade é de 50%. 

Analisa estes dados para ajudar o utilizador a compreender as condições de crescimento das plantas.
Fornece informações sobre:
1. Se as condições ambientais são adequadas para a planta em questão (girassol)
2. Recomendações para melhorar as condições de cultivo, se necessário
3. Interpretação das leituras do sensor
4. Sugestões de cuidados a ter com o girassol com base na temperatura e humidade atual em Lisboa

Como deve ser estruturada a resposta:
- Sê educativo e explica os conceitos claramente aos alunos 
- Adiciona muitos emojis amigáveis. 
- Responde em português europeu, podes tratar as pessoas por "tu".
"""

Agora... está na altura de fazer perguntas! No código em baixo consegues ver uma pergunta simples no `user_question`. Carrega no botão `Run`utilizando a pergunta que aqui está. De seguida, volta ao bloco de código acima para alterares as "ordens" que dás ao teu BasilGPT ou faz-lhe novas perguntas.

In [ ]:
# Function to call Azure OpenAI
def ask_garden_assistant(question):
    response = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_MODEL", "gpt-4o"),
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question}
        ],
        temperature=0.7,
        max_tokens=800
    )
    return response.choices[0].message.content

# Example usage
user_question = "Como é que está o meu girassol hoje?"
response = ask_garden_assistant(user_question)
display(Markdown(response))

Olá! 🌻 Vamos analisar as condições do teu girassol e ver como ele está a fazer!

### 1. Condições Ambientais
As condições ambientais que temos são as seguintes:
- **Temperatura**: 20°C
- **Humidade**: 50%
- **Tempo**: Ensolarado

**Adequação para o Girassol**: 
- A temperatura de **20°C** está um pouco abaixo da faixa ideal para girassóis, que é entre **21°C e 26°C**. No entanto, não te preocupes! Girassóis são bastante resilientes e conseguem tolerar temperaturas ligeiramente mais baixas. ☀️
- A humidade de **50%** está dentro do intervalo ideal (50% a 70%), o que é ótimo para o teu girassol! 🌿

### 2. Recomendações para Melhorar as Condições de Cultivo
- **Aumentar a Temperatura**: Se possível, durante o dia, podes colocar o girassol em locais onde receba mais calor, como junto a uma parede virada a sul. Isso ajudará a aumentar a temperatura. 🏡
- **Protecção contra Frio**: Se houver risco de temperaturas mais baixas à noite, usa um pano para cobrir a planta e protegê-la de eventuais quedas de temperatura. 

### 3. Interpretação das Leituras do Sensor
Os dados do sensor mostram:
- **Temperatura**: 16°C (média das leituras)
- **Humidade**: 0% (o que parece indicar um problema, pois não é uma leitura válida)
- **Luz**: 128 (a luz parece estar estável, mas não estás a obter leituras de humidade adequadas)

**Análise**: A temperatura média está a um nível que pode ser tolerável para o girassol, mas a leitura de humidade não está correta. Poderia ser um erro no sensor ou um problema de calibragem. Verifica se o sensor está a funcionar corretamente! 📉

### 4. Sugestões de Cuidados para o Girassol
- **Água**: Certifica-te de que o solo está consistentemente húmido, especialmente durante as fases iniciais de crescimento. Evita regar as folhas para prevenir doenças. 💧
- **Exposição ao Sol**: Os girassóis precisam de pelo menos 6 a 8 horas de luz solar direta. Aproveita o dia ensolarado e coloca-os numa área bem iluminada! 🌞
- **Espaçamento e Circulação de Ar**: Se tiveres mais girassóis, garante que têm espaço suficiente entre eles para permitir uma boa circulação de ar, o que é importante, especialmente em condições de humidade moderada. 

### Conclusão
No geral, as condições para o teu girassol estão a melhorar, embora precise de um bocadinho mais de calor. Com as devidas atenções e cuidados, ele deverá crescer saudável e feliz! 🌻✨ Se tiveres mais perguntas ou precisares de ajuda com algo específico, não hesites em perguntar! 😊